In [8]:
# import access to Google Drive files

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# import needed libraries

import os
import torch
import sys
import pandas as pd
import importlib

In [10]:
# create train-val-test sets for Ex2Vec training, as well as sequences for GRU4Rec training
!python /content/drive/MyDrive/JKU/practical_work/Practical-Work-AI/preprocess.py

Pre-processing dataset for Ex2Vec...
Saved processed.csv
Pre-processing dataset for GRU4Rec


In [3]:
# import custom code

# Append the directory containing 'data_sampler' and 'ex2vec' to Python's search path
sys.path.append('/content/drive/MyDrive/JKU/practical_work/Practical-Work-AI')

# imports modules for preparing data and for training/evaluating the ex2vec model
import data_sampler
from ex2vec import Ex2VecEngine

#import dir of gru4rec_pytorch module to python path in order to be able to access GRU4Rec model class to be able to load it
sys.path.append('/content/drive/MyDrive/JKU/practical_work/Practical-Work-AI/GRU4Rec_PyTorch_Fork')
from gru4rec_pytorch import GRU4RecModel
import evaluation as GRUeval

FileNotFoundError: [Errno 2] No such file or directory: 'data/processed.csv'

In [ ]:
!pip install tensorboardX

In [ ]:
# Check if gpu is available
if torch.cuda.is_available():
    device = torch.device('cuda')
    device_name = torch.cuda.get_device_name(device)
    print(f'Using GPU: {device_name}')
else:
    device = torch.device('cpu')
    print('Using CPU')

print(f'Current device: {device}')

In [ ]:
# check that deezer parameter file exists
param_file_path = '/content/drive/MyDrive/JKU/practical_work/Practical-Work-AI/GRU4Rec_PyTorch_Fork/paramfiles/deezer_paramfile.py'
assert os.path.isfile(param_file_path), f'Parameter file not found at {param_file_path}'

In [ ]:
# train Ex2Vec
!python /content/drive/MyDrive/JKU/practical_work/Practical-Work-AI/train.py

In [ ]:
# train GRU4Rec

"""
params: (from https://github.com/hidasib/GRU4Rec/blob/master/README.md)
  -t    Testset path
  -pf   Parameter file path
  -s    Path to save the state dict to
  -m    Calculate recall, MRR etc. at the given list length
  -ik   Item key
  -tk   Timestamp key
  -d    Device
"""

!python /content/drive/MyDrive/JKU/practical_work/GRU4Rec/run.py /content/drive/MyDrive/JKU/practical_work/Practical-Work-AI/data/seq_train.csv -t /content/drive/MyDrive/JKU/practical_work/Practical-Work-AI/data/seq_val.csv -pf /content/drive/MyDrive/JKU/practical_work/Practical-Work-AI/GRU4Rec_PyTorch_Fork/paramfiles/deezer_paramfile.py -s /content/drive/MyDrive/JKU/practical_work/Practical-Work-AI/models/GRU4Rec.pt -m 1 5 10 20 -ik "itemId" -tk "timestamp" -d cpu

In [ ]:
# extract item embeddings from GRU4Rec Model

model_loaded = torch.load('/content/drive/MyDrive/JKU/practical_work/Practical-Work-AI/models/GRU4Rec.pt')
item_embds = model_loaded.model.Wy.weight.data
item_embds.shape

In [ ]:
# retrain Ex2Vec with GRU4Rec item embeddings

In [ ]:
# compare performance